## Hugging Face & Deep Learning Tasks


## Task 1: Neural Network from Scratch
For this I tried to build a simpel neural net with numpy. Took some time to figure out the backpropagation but it kind of works. Not great results but still okay for scratch.

In [8]:
import numpy as np

# Simple 2-layer network
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_deriv(x):
    return x * (1 - x)

# dummy data
X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([[0],[1],[1],[0]])

np.random.seed(1)
weights0 = 2*np.random.random((2,4)) - 1
weights1 = 2*np.random.random((4,1)) - 1

for j in range(10000):
    layer0 = X
    layer1 = sigmoid(np.dot(layer0, weights0))
    layer2 = sigmoid(np.dot(layer1, weights1))
    layer2_error = y - layer2
    if j % 2000 == 0:
        print(f'Error: {np.mean(np.abs(layer2_error))}')
    layer2_delta = layer2_error * sigmoid_deriv(layer2)
    layer1_error = layer2_delta.dot(weights1.T)
    layer1_delta = layer1_error * sigmoid_deriv(layer1)
    weights1 += layer1.T.dot(layer2_delta)
    weights0 += layer0.T.dot(layer1_delta)

Error: 0.4995289647766772
Error: 0.051539874269874296
Error: 0.03268216266445581
Error: 0.02557644133425584
Error: 0.021587502901032785


## Task 2: Optimization (GPU/Cloud)
Tested a small transformer on Colab with GPU. Quite a bit faster than CPU, especially when I tried it with longer texts.

In [9]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('distilbert-base-uncased').to(device)
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, device=0 if device=='cuda' else -1)

result = classifier('this project is going better than I thought')
print(result)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


[{'label': 'LABEL_1', 'score': 0.5127564668655396}]


## Task 3: Software Quality
Just added a simple test to make sure tokenizer output has right stuff. In a real app I would use logging and error checks too.

In [10]:
def test_tokenizer():
    out = tokenizer('test input')
    assert 'input_ids' in out
    print('Tokenizer test passed')

test_tokenizer()

Tokenizer test passed


## Task 4: Object Detection
Used the DETR model from facebook. Works on default coco image, you can see the boxes. Not bad for one line code basically.

In [11]:
from transformers import DetrImageProcessor, DetrForObjectDetection
from PIL import Image, ImageDraw
import requests
import torch

image = Image.open(requests.get('http://images.cocodataset.org/val2017/000000039769.jpg', stream=True).raw)
processor = DetrImageProcessor.from_pretrained('facebook/detr-resnet-50')
model = DetrForObjectDetection.from_pretrained('facebook/detr-resnet-50')

inputs = processor(images=image, return_tensors='pt')
outputs = model(**inputs)
target_sizes = torch.tensor([image.size[::-1]])
results = processor.post_process_object_detection(outputs, target_sizes=target_sizes)[0]

draw = ImageDraw.Draw(image)
for score, label, box in zip(results['scores'], results['labels'], results['boxes']):
    if score > 0.9:
        draw.rectangle(list(box), outline='red', width=2)
        draw.text((box[0], box[1]), f'{model.config.id2label[label.item()]}: {round(score.item(), 3)}', fill='red')
image.show()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for conv1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for bn1.weight: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pass `assign=True` to assign items in the state dictionary to their corresponding key in the module instead of copying them in place?)
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:2397: UserWarning: for bn1.bias: copying from a non-meta parameter in the checkpoint to a meta parameter in the current model, which is a no-op. (Did you mean to pas

## Task 6: Fine-tune Language Model
Fine tuned BERT on emotion dataset. Accuracy wasn't too bad. Took a bit to get the trainer stuff working.

In [2]:
!pip install --upgrade datasets fsspec
!pip install --upgrade huggingface_hub # huggingface_hub is also a dependency

from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
import os # Import the os module to get the absolute path
import torch # Import torch

# Get the absolute path of the current directory and append the cache directory name
absolute_cache_dir = os.path.abspath('./hf_cache')

dataset = load_dataset('dair-ai/emotion', cache_dir=absolute_cache_dir)

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Modify the tokenize function to explicitly include the label
def tokenize(ex):
    tokenized_inputs = tokenizer(ex['text'], truncation=True, padding='max_length')
    # Add the label to the tokenized inputs dictionary
    tokenized_inputs['label'] = ex['label']
    return tokenized_inputs

tokenized = dataset.map(tokenize, batched=True)

# Set the format of the tokenized dataset for PyTorch
tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

args = TrainingArguments('test-train', evaluation_strategy='epoch', per_device_train_batch_size=8, num_train_epochs=1)
trainer = Trainer(model=model, args=args, train_dataset=tokenized['train'], eval_dataset=tokenized['validation'])

trainer.train()

  Using cached fsspec-2025.5.0-py3-none-any.whl.metadata (11 kB)


Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'